In [18]:
class PicoMotor:
    
    _MIN_SPEED = 50
    _MAX_SPEED = 100
    _MIN_DIFF = 10
    
    def __init__(self, serial_port):
        self.serial_port = serial_port
        
    def _send(self, left, right):
        cmd = f'MOTOR:left:{left}:right:{right}'.encode()
        serial_port.write(cmd)
        
    def _bounding_speed(self, speed):
        if speed == 0:
            return speed
        
        if abs(speed) < self._MIN_SPEED:
            print(f'[PicoMotor]: Lowest speed is {self._MIN_SPEED}, reset value {speed} to it.')
            return self._MIN_SPEED if speed > 0 else -self._MIN_SPEED
        
        if abs(speed) > self._MAX_SPEED:
            print(f'[PicoMotor]: Max speed is {self._MAX_SPEED}, reset value {speed} to it.')
            return self._MAX_SPEED if speed > 0 else -self._MAX_SPEED
        
        return speed
    
    def _get_diff_speed(self, speed, diff):
        if speed >= 0:
            return speed - diff
        
        return speed + diff
    
    def _is_need_change_to_rotate(self, speed, diff_speed):
        return abs(speed) == self._MIN_SPEED or (abs(speed) - abs(diff_speed)) < self._MIN_DIFF
        
    def go_straight(self, speed):
        print(f'[PicoMotor]: go_straight, speed={speed}')
        speed = self._bounding_speed(speed)
        self._send(speed, speed)

    def turn_left(self, speed, diff):
        print(f'[PicoMotor]: turn_left, speed={speed}, diff={diff}')
        if speed == 0:
            return
        
        speed = self._bounding_speed(speed)
        diff_speed = self._bounding_speed(self._get_diff_speed(speed, diff))
        
        if self._is_need_change_to_rotate(speed, diff_speed):
            print('[PicoMotor]: change to left_rotate')
            self.left_rotate()
        else:    
            self._send(diff_speed, speed)

    def turn_right(self, speed, diff):
        print(f'[PicoMotor]: turn_tight, speed={speed}, diff={diff}')
        if speed == 0:
            return
        
        speed = self._bounding_speed(speed)
        diff_speed = self._bounding_speed(self._get_diff_speed(speed, diff))
        
        if self._is_need_change_to_rotate(speed, diff_speed):
            print('[PicoMotor]: change to right_rotate')
            self.right_rotate()
        else:    
            self._send(speed, diff_speed)

    def left_rotate(self):
        print(f'[PicoMotor]: left_rotate')
        self._send(-self._MIN_SPEED, self._MIN_SPEED)
        
    def right_rotate(self):
        print(f'[PicoMotor]: right_rotate')     
        self._send(self._MIN_SPEED, -self._MIN_SPEED)
        
    def stop(self):
        self._send(0, 0)
    
    

In [ ]:
import serial

_DEFAULT_PORT = '/dev/ttyTHS1'
_BAUDRATE = 115200

serial_port = serial.Serial(
    port=port,
    baudrate=_BAUDRATE,
    bytesize=serial.EIGHTBITS,
    parity=serial.PARITY_NONE,
    stopbits=serial.STOPBITS_ONE,
)

In [15]:
class FakeSerial:
    def write(self, cmd):
        print(f'[FakeSerial]: {cmd}')
        
serial_port = FakeSerial()

In [20]:
m = PicoMotor(serial_port)
print('==== Test forward ====')
m.go_straight(100)
m.go_straight(60)
m.go_straight(20)
print('\n==== Test backward ====')
m.go_straight(-100)
m.go_straight(-60)
m.go_straight(-20)
print('\n==== Test left ====')
m.turn_left(100, 10)
m.turn_left(60, 10)
m.turn_left(60, 20)
m.turn_left(50, 10)
m.turn_left(59, 10)
m.turn_left(30, 10)
print('\n==== Test right ====')
m.turn_right(100, 10)
m.turn_right(60, 10)
m.turn_right(60, 20)
m.turn_right(50, 10)
m.turn_right(59, 10)
m.turn_right(30, 10)

==== Test forward ====
[PicoMotor]: go_straight, speed=100
[FakeSerial]: b'MOTOR:left:100:right:100'
[PicoMotor]: go_straight, speed=60
[FakeSerial]: b'MOTOR:left:60:right:60'
[PicoMotor]: go_straight, speed=20
[PicoMotor]: Lowest speed is 50, reset value 20 to it.
[FakeSerial]: b'MOTOR:left:50:right:50'

==== Test backward ====
[PicoMotor]: go_straight, speed=-100
[FakeSerial]: b'MOTOR:left:-100:right:-100'
[PicoMotor]: go_straight, speed=-60
[FakeSerial]: b'MOTOR:left:-60:right:-60'
[PicoMotor]: go_straight, speed=-20
[PicoMotor]: Lowest speed is 50, reset value -20 to it.
[FakeSerial]: b'MOTOR:left:-50:right:-50'

==== Test left ====
[PicoMotor]: turn_left, speed=100, diff=10
[FakeSerial]: b'MOTOR:left:90:right:100'
[PicoMotor]: turn_left, speed=60, diff=10
[FakeSerial]: b'MOTOR:left:50:right:60'
[PicoMotor]: turn_left, speed=60, diff=20
[PicoMotor]: Lowest speed is 50, reset value 40 to it.
[FakeSerial]: b'MOTOR:left:50:right:60'
[PicoMotor]: turn_left, speed=50, diff=10
[PicoMotor